In [1]:
# 필요한 라이브러리 import
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import requests
from selenium.webdriver.common.keys import Keys
import time
import warnings
warnings.filterwarnings('ignore')

In [2]:
base_url = "https://www.bbc.com"
start_url = "https://www.bbc.com/news/world/europe"

# 해당 페이지에서 각 기사들의 url 뽑아오는 함수
def get_articles_urls(soup):
    return [base_url + li.a["href"] for li in soup.select("li.lx-stream__post-container")], [li.find('article').find('div').find('div').find('time').find_all('span')[1].text for li in soup.select("li.lx-stream__post-container")]


# 해당 페이지 기사들의 정보 뽑아오는 코드
def get_page_articles(driver):
    driver_source = driver.page_source
    soup = BeautifulSoup(driver_source, 'html.parser')
    article_urls, article_times = get_articles_urls(soup)

    article_titles = []
    article_time_save = []
    article_contents = []
    article_relateds = []
    for article_url, article_time in zip(article_urls, article_times):
        article_title, article_content, article_related = extract_single_article_content(article_url)
        if article_title and article_content:
            article_titles.append(article_title)
            article_time_save.append(article_time)
            article_contents.append(article_content)
            article_relateds.append(article_related)
    return article_titles, article_time_save, article_contents, article_relateds, soup


# 한 기사의 제목, 본문, 태그들 가져오는 함수
def extract_single_article_content(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    
    # 기사 제목 가져오기
    title = soup.select_one("h1")
    if title:
        title = title.get_text(strip=True)
    else:
        return None, None, None
    
    # 기사 내용 가져오기
    paragraphs = []
    p_elements = soup.select("div.ssrcss-7uxr49-RichTextContainer.e5tfeyi1 > p")
    
    if not p_elements: # 비어있는 경우를 확인하여 에러를 방지합니다.
        return None, None, None
    
    last_p_element = p_elements[-1]
    # div.ssrcss-7uxr49-RichTextContainer.e5tfeyi1 클래스 하위의 p 요소들을 가져오기
    for p in p_elements:
        # 마지막 문단의 a, i에 태그가 나올 땐 날려버려야 하는 경우가 있다.
        if p == last_p_element:
            # p 요소 안에 있는 모든 a와 i 태그 삭제
            for tag in p.find_all(["i", "a"]):
                tag.decompose()
        # p 요소의 텍스트만 추출하여 paragraphs에 추가
        paragraphs.append(p.text.strip())

    content = "\n".join(paragraphs)
    
    # 기사 태그 가져오기
    tags = soup.select('div.ssrcss-1qmkvfu-TopicListWrapper.etw6iwl1 > div.ssrcss-1szabdv-StyledTagContainer.ed0g1kj1 > div.ssrcss-17ehax8-Cluster.e1ihwmse1 > ul.ssrcss-1ujonwb-ClusterItems.e1ihwmse0 > li')
    if not tags:
        return None, None, None
    
    related = ', '.join([tag.get_text() for tag in tags])
       

    return title, content, related



In [3]:
# 크롬 드라이버 설치, 연결
s = Service("D:\chromedriver.exe")
driver = webdriver.Chrome(service=s)

# 브라우저 화면 크기 변경하기
driver.maximize_window()

# 웹 페이지 열기
url = "https://www.bbc.com/news/world/europe"
driver.get(url)

# 데이터 프레임 초기화
df_europe = pd.DataFrame(columns=["Title", "Time", "Content", "Related"])

# title과 content를 가져오기 위해 get_page_articles 함수 호출
article_titles, article_time_save, article_contents, article_relateds, soup = get_page_articles(driver)

body = driver.find_elements('css selector', 'body')[0]
for i in range(17):
    body.send_keys(Keys.PAGE_DOWN)
    
btn = driver.find_elements('css selector', 'div > div.gel-icon.gel-icon--next')[0]

while True:
    try:
        # 데이터 프레임에 저장
        for title, time_save, content, related in zip(article_titles, article_time_save, article_contents, article_relateds):
            if title in df_europe:
                continue
            else:
                df_europe = df_europe.append({"Title": title, "Time": time_save, "Content": content, "Related": related}, ignore_index=True)
        
        # 500개 까지만 모으기
        if len(df_europe) > 500:
                break

        print('모인 기사 수: ' + str(len(df_europe)))

        time.sleep(1)

        btn.click()

        # 페이지를 변경한 후에도 다음 뉴스 목록을 가져올 수 있도록 변경된 페이지에 대한 soup 객체 생성
        driver_source = driver.page_source
        page_soup = BeautifulSoup(driver_source, 'html.parser')

        # 변경된 페이지에 대한 정보를 크롤링하고 순환하는 데 사용
        article_titles, article_time_save, article_contents, aricle_relateds, soup = get_page_articles(driver)

    except:
        btn.click()
    

# 웹 드라이버 종료
driver.quit()

# 데이터 프레임 출력
df_europe

모인 기사 수: 17
모인 기사 수: 34


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=119.0.6045.159)
Stacktrace:
	GetHandleVerifier [0x00007FF69BC282B2+55298]
	(No symbol) [0x00007FF69BB95E02]
	(No symbol) [0x00007FF69BA505AB]
	(No symbol) [0x00007FF69BA30038]
	(No symbol) [0x00007FF69BAB6BC7]
	(No symbol) [0x00007FF69BACA15F]
	(No symbol) [0x00007FF69BAB1E83]
	(No symbol) [0x00007FF69BA8670A]
	(No symbol) [0x00007FF69BA87964]
	GetHandleVerifier [0x00007FF69BFA0AAB+3694587]
	GetHandleVerifier [0x00007FF69BFF728E+4048862]
	GetHandleVerifier [0x00007FF69BFEF173+4015811]
	GetHandleVerifier [0x00007FF69BCC47D6+695590]
	(No symbol) [0x00007FF69BBA0CE8]
	(No symbol) [0x00007FF69BB9CF34]
	(No symbol) [0x00007FF69BB9D062]
	(No symbol) [0x00007FF69BB8D3A3]
	BaseThreadInitThunk [0x00007FFD2A7E257D+29]
	RtlUserThreadStart [0x00007FFD2BF6AA58+40]


In [4]:
# 중복 기사 제거
df_europe = df_europe.drop_duplicates()

In [5]:
df_europe

,Title,Time,Content,Related
0,Berlin on edge for Erdogan after fierce Israel...,10:30,No speeches in front of cheering crowds. No jo...,"Israel-Gaza war, Germany, Recep Tayyip Erdogan"
1,Swimming rivers and faking illness to escape U...,9:30,"Nearly 20,000 men have fled Ukraine since the ...","Russia-Ukraine war, Military, Ukraine"
2,Two dead and child seriously injured in road c...,7:23,Two people have died and a child has sustained...,"Europe, Republic of Ireland, Waterford"
3,Spain's Pedro SÃ¡nchez wins new term as PM aft...,4:30,"After weeks of haggling, Socialist leader Pedr...","Europe, Spain, Pedro Sanchez"
4,Ukraine in maps: Tracking the war with Russia,2:50,Ukraine's counter-offensive has now been under...,"Russia-Ukraine war, Volodymyr Zelensky, Russia..."
...,...,...,...,...
324,Luton Airport shuttle buses join Ukraine war e...,13:48 7 Sep,Three former airport shuttle buses have been d...,"Russia-Ukraine war, Russia, Ukraine, Ukrainian..."
325,UK expected to re-join Horizon science scheme,13:39 7 Sep,The UK is expected to re-join the EU's flagshi...,"Volcanoes, Iceland"
326,Migrants bundled out of Europe lose court chal...,0:57 7 Sep,The EU's top court has dismissed a case agains...,"Mark Rutte, Netherlands"
327,Ukraine war: Romania reveals Russian drone par...,0:16 7 Sep,Romania's defence minister has said that remai...,"Europe, Israel & the Palestinians, Republic of..."


In [6]:
df_europe.to_excel('./data/europe.xlsx', index=False)